In [ ]:
import sys
sys.path.append('../input/huggingface-accelerate')
sys.path.append('../input/transformershuggingface/transformer_repo')

In [ ]:
model_checkpoint='../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2'

In [ ]:
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq

In [ ]:
import numpy as np
import collections
import pandas as pd
import gc
import os
import sys
from IPython.display import FileLink
%env WANDB_DISABLED=True
import torch
import datasets as d
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader
from src.accelerate import Accelerator
import math
import os
import random
from tqdm.auto import tqdm
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    default_data_collator,
    get_scheduler,
    SchedulerType,
    set_seed)


In [ ]:
# set the seed
seed=124
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
set_seed(seed)

In [ ]:
# load  data
import pandas as pd
train_df=pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')
test_df=pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
submission_df=pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv')
squad=pd.read_csv('../input/squadhindi/xquad.csv')
mlqa=pd.read_csv('../input/mlqa-hindi-processed/mlqa_hindi.csv')

In [ ]:
squad['id']=['sar'+str(item) for item in range(5427,5427+squad.shape[0])]
mlqa['id']=['sar'+str(item) for item in range(mlqa.shape[0])]
# merge train_df and squad
train_df=pd.concat([train_df,squad,mlqa])

In [ ]:
def get_answers_as_dict(r):
    start = r[0]
    text = r[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

train_df['answers'] = train_df[['answer_start', 'answer_text']].apply(get_answers_as_dict, axis=1)

train_df.reset_index(drop=True,inplace=True)

In [ ]:
# hyper parameters
max_length = 384
doc_stride = 128
eval_batch_size = 8

In [ ]:
def prepare_test_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
def postprocess_qa_predictions(
    examples,
    features,
    predictions,
    version_2_with_negative: bool = False,
    n_best_size: int = 20,
    max_answer_length: int = 30,
    null_score_diff_threshold: float = 0.0):
    
    assert len(predictions) == 2, "`predictions` should be a tuple with two elements (start_logits, end_logits)."
    all_start_logits, all_end_logits = predictions

    assert len(predictions[0]) == len(features), f"Got {len(predictions[0])} predictions and {len(features)} features."

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()
    
    
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_prediction = None
        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]
            # Optional `token_is_max_context`, if provided we will remove answers that do not have the maximum context
            # available in the current feature.
            token_is_max_context = features[feature_index].get("token_is_max_context", None)

            # Update minimum null prediction.
            feature_null_score = start_logits[0] + end_logits[0]
            if min_null_prediction is None or min_null_prediction["score"] > feature_null_score:
                min_null_prediction = {
                    "offsets": (0, 0),
                    "score": feature_null_score,
                    "start_logit": start_logits[0],
                    "end_logit": end_logits[0],
                }

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    # Don't consider answer that don't have the maximum context available (if such information is
                    # provided).
                    if token_is_max_context is not None and not token_is_max_context.get(str(start_index), False):
                        continue
                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                            "score": start_logits[start_index] + end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )
        if version_2_with_negative:
            # Add the minimum null prediction
            prelim_predictions.append(min_null_prediction)
            null_score = min_null_prediction["score"]

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:n_best_size]

        # Add back the minimum null prediction if it was removed because of its low score.
        if version_2_with_negative and not any(p["offsets"] == (0, 0) for p in predictions):
            predictions.append(min_null_prediction)

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0] : offsets[1]]

        # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
        # failure.
        if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0, "end_logit": 0.0, "score": 0.0})

        # Compute the softmax of all scores (we do it with numpy to stay independent from torch/tf in this file, using
        # the LogSumExp trick).
        scores = np.array([pred.pop("score") for pred in predictions])
        exp_scores = np.exp(scores - np.max(scores))
        probs = exp_scores / exp_scores.sum()

        # Include the probabilities in our predictions.
        for prob, pred in zip(probs, predictions):
            pred["probability"] = prob

        # Pick the best prediction. If the null answer is not possible, this is easy.
        if not version_2_with_negative:
            all_predictions[example["id"]] = predictions[0]["text"]
        else:
            # Otherwise we first need to find the best non-empty prediction.
            i = 0
            while predictions[i]["text"] == "":
                i += 1
            best_non_null_pred = predictions[i]

            # Then we compare to the null prediction using the threshold.
            score_diff = null_score - best_non_null_pred["start_logit"] - best_non_null_pred["end_logit"]
            if score_diff > null_score_diff_threshold:
                all_predictions[example["id"]] = ""
            else:
                all_predictions[example["id"]] = best_non_null_pred["text"]

        

    return all_predictions

In [ ]:
# Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
def create_and_fill_np_array(start_or_end_logits, dataset, max_len):

        step = 0
        # create a numpy array and fill it with -100.
        logits_concat = np.full((len(dataset), max_len), -100, dtype=np.float64)
        # Now since we have create an array now we will populate it with the outputs gathered using accelerator.gather
        for i, output_logit in enumerate(start_or_end_logits):  # populate columns
            # We have to fill it such that we have to take the whole tensor and replace it on the newly created array
            # And after every iteration we have to change the step

            batch_size = output_logit.shape[0]
            cols = output_logit.shape[1]

            if step + batch_size < len(dataset):
                logits_concat[step : step + batch_size, :cols] = output_logit
            else:
                logits_concat[step:, :cols] = output_logit[: len(dataset) - step]

            step += batch_size

        return logits_concat

In [ ]:
def logit_predictions(test_data,checkpoint,accelerator):
  config = AutoConfig.from_pretrained(checkpoint)
  model = AutoModelForQuestionAnswering.from_pretrained(checkpoint,config=config)
  device=torch.device("cuda")
  model.to(device)
  test_dataset_for_model = test_data.remove_columns(["example_id", "offset_mapping"])
  test_dataloader = DataLoader(
            test_dataset_for_model, collate_fn=default_data_collator, batch_size=eval_batch_size)

  all_start_logits = []
  all_end_logits = []
  for step, batch in enumerate(tqdm(test_dataloader)):
        with torch.no_grad():
            for key,value in batch.items():
                batch[key]=value.to(device)
            outputs = model(**batch)
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits

                
            all_start_logits.append(accelerator.gather(start_logits).cpu().numpy())
            all_end_logits.append(accelerator.gather(end_logits).cpu().numpy())

  max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor
  # concatenate the numpy array
  start_logits_concat = create_and_fill_np_array(all_start_logits, test_data, max_len)
  end_logits_concat = create_and_fill_np_array(all_end_logits, test_data, max_len)

  # delete the list of numpy arrays
  del all_start_logits
  del all_end_logits
  del model
  del config
  gc.collect()

  return start_logits_concat, end_logits_concat

In [ ]:
test_examples=d.Dataset.from_pandas(test_df)
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)  
test_dataset = test_examples.map(
            prepare_test_features,
            batched=True,
            remove_columns=test_examples.column_names
        )
accelerator=Accelerator()
checkpoints_dir=['../input/chaiikfold01234/fold0/fold0','../input/chaiikfold01234/fold1/fold1',
                 '../input/chaiikfold01234/fold2/fold2',
                '../input/chaiikfold01234/fold3/fold3','../input/chaiikfold01234/fold4/fold4']
output_numpy=[0.0,0.0]
for checkpoint in checkpoints_dir:
  output=logit_predictions(test_dataset,checkpoint,accelerator)
  output_numpy[0]+=output[0]
  output_numpy[1]+=output[1]
output_numpy=(output_numpy[0]/5,output_numpy[1]/5)
final_predictions = postprocess_qa_predictions(test_examples,test_dataset,output_numpy)



In [ ]:
# cleaned predictions
test_df['PredictionString']=test_df['id'].apply(lambda r:final_predictions[r])
bad_starts = [".", ",", "(", ")", "-", "–",  ",", ";"]
bad_endings = ["...", "-", "(", ")", "–", ",", ";"]

tamil_ad = "கி.பி"
tamil_bc = "கி.மு"
tamil_km = "கி.மீ"
hindi_ad = "ई"
hindi_bc = "ई.पू"


cleaned_preds = []
for pred, context in test_df[["PredictionString", "context"]].to_numpy():
    if pred == "":
        cleaned_preds.append(pred)
        continue
    while any([pred.startswith(y) for y in bad_starts]):
        pred = pred[1:]
    while any([pred.endswith(y) for y in bad_endings]):
        if pred.endswith("..."):
            pred = pred[:-3]
        else:
            pred = pred[:-1]
    if pred.endswith("..."):
            pred = pred[:-3]
    
    if any([pred.endswith(tamil_ad), pred.endswith(tamil_bc), pred.endswith(tamil_km), pred.endswith(hindi_ad), pred.endswith(hindi_bc)]) and pred+"." in context:
        pred = pred+"."
        
    cleaned_preds.append(pred)

In [ ]:
# submission file
sub=pd.DataFrame({'id':test_df['id']})
sub['PredictionString']=cleaned_preds
sub

In [ ]:
sub.to_csv("submission.csv",index=False)